# SO101 机械臂设置和标定教程

本教程将指导您完成 SO101 机械臂的完整设置流程，包括：

1. ✅ 查找 USB 端口
2. ✅ 设置电机 ID 和波特率
3. ✅ 标定主臂 (Leader Arm)
4. ✅ 标定从臂 (Follower Arm)
5. ✅ 测试主从臂通信

**环境要求**:
- Ubuntu 22.04
- Python 3.10+
- LeRobot 已安装 (包括 feetech 驱动)

**硬件要求**:
- SO101 主臂 (Leader) x 1
- SO101 从臂 (Follower) x 1
- USB 线缆 x 2
- 电源适配器 x 2

---
## 📦 步骤 0: 导入必要的库

In [ ]:
# 标准库
import os
import sys
import time
from pathlib import Path

# LeRobot 相关
from lerobot.robots.so101_follower import SO101Follower, SO101FollowerConfig
from lerobot.teleoperators.so101_leader import SO101Leader, SO101LeaderConfig

# 辅助函数
import subprocess

print("✅ 库导入成功！")
print(f"当前工作目录: {os.getcwd()}")

---
## 🔍 步骤 1: 查找 USB 端口

首先，我们需要找到主臂和从臂分别对应的 USB 端口。

### 方法 1: 使用 LeRobot 内置工具

In [ ]:
# 在 Linux 上，首先需要给予端口权限
# 注意：这需要 sudo 权限，可能需要在终端手动运行

print("⚠️ 请在终端运行以下命令给予 USB 端口权限：")
print("")
print("sudo chmod 666 /dev/ttyACM0")
print("sudo chmod 666 /dev/ttyACM1")
print("")
print("然后运行以下命令查找端口：")
print("lerobot-find-port")
print("")
print("📋 按照提示，拔掉一个臂的 USB 线，程序会告诉您端口地址。")

### 方法 2: 手动查找端口

In [ ]:
# 列出所有 USB 串口设备
!ls -l /dev/ttyACM* /dev/ttyUSB* 2>/dev/null || echo "未找到 USB 设备"

# 或者使用 dmesg 查看最近连接的设备
print("\n最近连接的 USB 设备:")
!dmesg | grep tty | tail -10

### 📝 记录您的端口

完成上述步骤后，请在下方填写您找到的端口：

In [ ]:
# ⚠️ 请根据您的实际情况修改以下端口
# 示例: "/dev/ttyACM0" 或 "/dev/tty.usbmodem585A0076841" (Mac)

FOLLOWER_PORT = "/dev/ttyACM0"  # 从臂端口
LEADER_PORT = "/dev/ttyACM1"    # 主臂端口

# 给机械臂命名 (用于保存标定文件)
FOLLOWER_ID = "my_so101_follower"
LEADER_ID = "my_so101_leader"

print(f"✅ 从臂端口: {FOLLOWER_PORT}")
print(f"✅ 主臂端口: {LEADER_PORT}")
print(f"✅ 从臂 ID: {FOLLOWER_ID}")
print(f"✅ 主臂 ID: {LEADER_ID}")

---
## ⚙️ 步骤 2: 设置从臂电机

为每个电机设置唯一的 ID 和波特率。

**重要说明**:
- 这个步骤只需要做**一次**
- 如果您的机械臂已经设置过，可以跳过此步骤
- 设置时需要**逐个连接电机**，不要同时连接多个

### 从臂电机 ID 映射

| 电机名称 | ID | 说明 |
|---------|-----|------|
| shoulder_pan | 1 | 肩部旋转 |
| shoulder_lift | 2 | 肩部抬升 |
| elbow_flex | 3 | 肘部弯曲 |
| wrist_flex | 4 | 腕部弯曲 |
| wrist_roll | 5 | 腕部旋转 |
| gripper | 6 | 夹爪 |

In [ ]:
# 方法 1: 使用 API (推荐)
# 注意：这会启动一个交互式流程，需要您按照提示操作

print("🔧 开始设置从臂电机...")
print("")
print("⚠️ 请按照以下顺序连接电机：")
print("   1. gripper (夹爪)")
print("   2. wrist_roll (腕部旋转)")
print("   3. wrist_flex (腕部弯曲)")
print("   4. elbow_flex (肘部弯曲)")
print("   5. shoulder_lift (肩部抬升)")
print("   6. shoulder_pan (肩部旋转)")
print("")
print("每次只连接一个电机到控制板！")
print("")

# 创建从臂配置
follower_config = SO101FollowerConfig(
    port=FOLLOWER_PORT,
    id=FOLLOWER_ID,
)

# 创建从臂实例
follower = SO101Follower(follower_config)

# 运行设置流程
# 注意：这是一个交互式流程，会在控制台提示您
print("⚠️ 准备开始设置流程，请确保：")
print("   1. 电源已接通")
print("   2. USB 已连接")
print("   3. 只连接了一个电机 (gripper)")
print("")
input("按 Enter 继续...")

follower.setup_motors()

print("")
print("✅ 从臂电机设置完成！")

### 或者使用命令行工具

In [ ]:
# 方法 2: 使用命令行 (需要在终端运行)
# 注意：此单元格仅显示命令，不会实际执行

command = f"""
lerobot-setup-motors \\
    --robot.type=so101_follower \\
    --robot.port={FOLLOWER_PORT}
"""

print("📋 请在终端运行以下命令：")
print(command)
print("")
print("⚠️ 注意：在 Jupyter Notebook 中运行交互式命令可能有问题")
print("建议直接在终端运行上述命令")

---
## ⚙️ 步骤 3: 设置主臂电机

主臂的设置流程与从臂相同，但使用不同的齿轮比。

### 主臂电机 ID 和齿轮比

| 电机名称 | ID | 齿轮比 | 说明 |
|---------|-----|--------|------|
| shoulder_pan | 1 | 1/191 | 肩部旋转 |
| shoulder_lift | 2 | 1/345 | 肩部抬升 |
| elbow_flex | 3 | 1/191 | 肘部弯曲 |
| wrist_flex | 4 | 1/147 | 腕部弯曲 |
| wrist_roll | 5 | 1/147 | 腕部旋转 |
| gripper | 6 | 1/147 | 夹爪 |

In [ ]:
print("🔧 开始设置主臂电机...")
print("")
print("⚠️ 请按照相同顺序连接主臂电机")
print("")

# 创建主臂配置
leader_config = SO101LeaderConfig(
    port=LEADER_PORT,
    id=LEADER_ID,
)

# 创建主臂实例
leader = SO101Leader(leader_config)

# 运行设置流程
print("⚠️ 准备开始设置主臂，请确保：")
print("   1. 主臂电源已接通")
print("   2. 主臂 USB 已连接")
print("   3. 只连接了一个电机 (gripper)")
print("")
input("按 Enter 继续...")

leader.setup_motors()

print("")
print("✅ 主臂电机设置完成！")

---
## 📏 步骤 4: 标定从臂

标定是非常重要的步骤，它确保：
- 主臂和从臂在相同物理位置时具有相同的数值
- 训练的模型可以在不同机器人间迁移

### 标定流程：
1. 将机械臂移动到所有关节都在中间位置
2. 按 Enter
3. 依次移动每个关节到最大和最小位置

In [ ]:
print("📏 开始标定从臂...")
print("")
print("⚠️ 标定前准备：")
print("   1. 确保所有电机已连接 (使用菊花链连接)")
print("   2. 第一个电机 (shoulder_pan, ID=1) 连接到控制板")
print("   3. 电源和 USB 都已连接")
print("")
input("按 Enter 继续...")

# 创建从臂实例
follower_config = SO101FollowerConfig(
    port=FOLLOWER_PORT,
    id=FOLLOWER_ID,
)
follower = SO101Follower(follower_config)

# 连接（不自动标定）
follower.connect(calibrate=False)

print("")
print("🎯 请将从臂移动到中间位置（所有关节在运动范围中点）")
print("")
print("参考姿势：")
print("   - 手臂自然伸展")
print("   - 不要在运动极限位置")
print("   - 夹爪半开")
print("")
input("准备好后按 Enter 开始标定...")

# 运行标定
follower.calibrate()

# 断开连接
follower.disconnect()

print("")
print("✅ 从臂标定完成！")
print(f"📁 标定文件已保存到: ~/.cache/lerobot/calibration/")

### 验证标定文件

In [ ]:
# 查看标定文件
calibration_dir = Path.home() / ".cache/lerobot/calibration"
print(f"📁 标定文件位置: {calibration_dir}")
print("")

if calibration_dir.exists():
    calibration_files = list(calibration_dir.glob(f"*{FOLLOWER_ID}*"))
    print(f"找到 {len(calibration_files)} 个从臂标定文件：")
    for f in calibration_files:
        print(f"  - {f.name}")
else:
    print("⚠️ 标定目录不存在")

---
## 📏 步骤 5: 标定主臂

对主臂执行相同的标定流程。

In [ ]:
print("📏 开始标定主臂...")
print("")

# 创建主臂实例
leader_config = SO101LeaderConfig(
    port=LEADER_PORT,
    id=LEADER_ID,
)
leader = SO101Leader(leader_config)

# 连接（不自动标定）
leader.connect(calibrate=False)

print("")
print("🎯 请将主臂移动到中间位置")
print("提示：主臂应该和从臂处于相同的物理姿势")
print("")
input("准备好后按 Enter 开始标定...")

# 运行标定
leader.calibrate()

# 断开连接
leader.disconnect()

print("")
print("✅ 主臂标定完成！")

# 查看主臂标定文件
leader_files = list(calibration_dir.glob(f"*{LEADER_ID}*"))
print(f"找到 {len(leader_files)} 个主臂标定文件：")
for f in leader_files:
    print(f"  - {f.name}")

---
## 🧪 步骤 6: 测试主从臂通信

验证主臂和从臂能够正常通信，从臂能跟随主臂运动。

In [ ]:
print("🧪 测试主从臂通信...")
print("")

# 重新创建配置
follower_config = SO101FollowerConfig(
    port=FOLLOWER_PORT,
    id=FOLLOWER_ID,
)

leader_config = SO101LeaderConfig(
    port=LEADER_PORT,
    id=LEADER_ID,
)

# 连接（使用标定）
follower = SO101Follower(follower_config)
leader = SO101Leader(leader_config)

follower.connect()
leader.connect()

print("✅ 主从臂已连接")
print("")
print("🎮 开始遥操作测试 (持续 30 秒)...")
print("")
print("⚠️ 请移动主臂，观察从臂是否跟随")
print("")

try:
    # 遥操作循环
    start_time = time.time()
    test_duration = 30  # 秒
    
    while time.time() - start_time < test_duration:
        # 读取主臂状态
        leader_obs = leader.get_observation()
        
        # 发送到从臂
        follower.send_action(leader_obs)
        
        # 显示进度
        elapsed = int(time.time() - start_time)
        remaining = test_duration - elapsed
        print(f"\r⏱️  剩余时间: {remaining} 秒", end="", flush=True)
        
        # 控制频率 (约 50 Hz)
        time.sleep(0.02)
    
    print("\n")
    print("✅ 遥操作测试完成！")
    
except KeyboardInterrupt:
    print("\n")
    print("⚠️ 用户中断测试")
    
finally:
    # 断开连接
    follower.disconnect()
    leader.disconnect()
    print("✅ 已断开连接")

---
## 📊 步骤 7: 读取关节状态

验证可以正确读取关节位置和速度。

In [ ]:
print("📊 读取关节状态...")
print("")

# 连接从臂
follower_config = SO101FollowerConfig(
    port=FOLLOWER_PORT,
    id=FOLLOWER_ID,
)
follower = SO101Follower(follower_config)
follower.connect()

# 读取观测值
obs = follower.get_observation()

print("从臂关节状态：")
print("")
print(f"关节位置: {obs}")
print("")

# 关节名称
joint_names = [
    "shoulder_pan",
    "shoulder_lift",
    "elbow_flex",
    "wrist_flex",
    "wrist_roll",
    "gripper"
]

print("各关节详细信息：")
for i, name in enumerate(joint_names):
    if i < len(obs):
        print(f"  {name:15s}: {obs[i]:.3f} rad")

# 断开连接
follower.disconnect()

print("")
print("✅ 状态读取成功！")

---
## ✅ 设置完成！

恭喜！您已经成功完成了 SO101 机械臂的设置和标定。

### 后续步骤：

1. **数据采集**: 使用 `02_so101_data_collection.ipynb`
2. **训练模型**: 使用 `03_so101_smolvla_training.ipynb`
3. **部署推理**: 使用训练好的模型控制机械臂

### 标定文件位置：

```
~/.cache/lerobot/calibration/
```

**重要提示**:
- 标定文件会在下次连接时自动加载
- 如果更换电机或机械结构，需要重新标定
- 建议备份标定文件

---
## 🔧 故障排除

### 问题 1: 找不到端口

```bash
# 检查 USB 设备
ls -l /dev/ttyACM*

# 给予权限
sudo chmod 666 /dev/ttyACM0
```

### 问题 2: 电机连接失败

检查：
1. 电源是否接通
2. USB 线是否连接
3. 3-pin 电机线是否插好
4. Waveshare 板跳线是否设置到 B 通道

### 问题 3: 标定失败

1. 确保所有电机都已连接
2. 确保电机 ID 已正确设置
3. 尝试重新运行标定流程

### 获取帮助

- [LeRobot Discord](https://discord.com/invite/s3KuuzsPFb)
- [GitHub Issues](https://github.com/huggingface/lerobot/issues)